# Test Conversation with Tools

Test tool calling in story scenarios with:
1. **calculator_pemdas**: Math calculations with PEMDAS order
2. **random_number_given_min_max**: Random integer generation
3. **random_choice_with_weights**: Weighted random selection

Features:
- Uses OpenRouter Chat Completions API with tool calling
- High reasoning effort for better decision making
- Sample story conversation included
- Full tool execution loop

In [ ]:
import os
import json
import ast
import operator
import random
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("OPENROUTER_API_KEY")

# ============================================================
# TOOL DEFINITIONS
# ============================================================
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculator_pemdas",
            "description": """Evaluates mathematical expressions following PEMDAS order of operations (Parentheses, Exponents, Multiplication/Division, Addition/Subtraction).

Usage examples:
- Calculate damage: "15 * 2 + 3" returns 33
- Subtract resources: "45 - 3 - 2 - 2" returns 38
- Complex calculation: "(10 + 5) * 2 ** 2" returns 60
- Division: "100 / 4" returns 25.0

Supported operators: +, -, *, /, //, %, ** (power), parentheses for grouping""",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate (e.g., '15 * 2 + 3')"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "random_number_given_min_max",
            "description": """Generates a random integer between min and max (inclusive).

Usage examples:
- Roll d20: min=1, max=20 returns random number 1-20
- Roll d6: min=1, max=6 returns random number 1-6
- Percentage: min=1, max=100 returns random number 1-100
- Damage roll: min=5, max=15 returns random number 5-15

Use this for dice rolls, random events, or any situation requiring a random number in a range.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "min": {
                        "type": "integer",
                        "description": "Minimum value (inclusive)"
                    },
                    "max": {
                        "type": "integer",
                        "description": "Maximum value (inclusive)"
                    }
                },
                "required": ["min", "max"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "random_choice_with_weights",
            "description": """Selects a random choice from a list with optional probability weights.

Usage examples:
- Weighted outcome: choices=["success", "failure"], weights=[0.7, 0.3] returns "success" 70% of time
- Equal probability: choices=["north", "south", "east", "west"] returns each 25% of time
- NPC attitude: choices=["friendly", "neutral", "hostile"], weights=[0.2, 0.5, 0.3]
- Loot drop: choices=["legendary", "rare", "common"], weights=[0.05, 0.25, 0.7]

If weights not provided, all choices have equal probability. Weights should sum to 1.0.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "choices": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of possible choices to select from"
                    },
                    "weights": {
                        "type": "array",
                        "items": {"type": "number"},
                        "description": "Optional probability weights for each choice (should sum to 1.0)"
                    }
                },
                "required": ["choices"]
            }
        }
    }
]

# ============================================================
# TOOL EXECUTION FUNCTIONS
# ============================================================
def execute_calculator_pemdas(expression):
    """Safely evaluate a mathematical expression using AST parsing."""
    # Allowed operators
    operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.FloorDiv: operator.floordiv,
        ast.Mod: operator.mod,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
    }
    
    def eval_node(node):
        if isinstance(node, ast.Num):  # Number
            return node.n
        elif isinstance(node, ast.Constant):  # Python 3.8+ uses Constant
            return node.value
        elif isinstance(node, ast.BinOp):  # Binary operation
            left = eval_node(node.left)
            right = eval_node(node.right)
            op = operators.get(type(node.op))
            if op is None:
                raise ValueError(f"Unsupported operator: {type(node.op).__name__}")
            # Check for division by zero
            if isinstance(node.op, (ast.Div, ast.FloorDiv, ast.Mod)) and right == 0:
                raise ZeroDivisionError("Division by zero")
            return op(left, right)
        elif isinstance(node, ast.UnaryOp):  # Unary operation (e.g., -5)
            operand = eval_node(node.operand)
            op = operators.get(type(node.op))
            if op is None:
                raise ValueError(f"Unsupported operator: {type(node.op).__name__}")
            return op(operand)
        else:
            raise ValueError(f"Unsupported expression: {type(node).__name__}")
    
    try:
        # Parse the expression
        tree = ast.parse(expression, mode='eval')
        result = eval_node(tree.body)
        return {"result": result, "expression": expression}
    except ZeroDivisionError:
        return {"error": "Division by zero", "expression": expression}
    except (ValueError, SyntaxError) as e:
        return {"error": f"Invalid expression: {str(e)}", "expression": expression}
    except Exception as e:
        return {"error": f"Calculation error: {str(e)}", "expression": expression}

def execute_random_number_given_min_max(min_val, max_val):
    """Generate random integer between min and max (inclusive)."""
    result = random.randint(min_val, max_val)
    return {"result": result, "min": min_val, "max": max_val}

def execute_random_choice_with_weights(choices, weights=None):
    """Select random choice with optional weights."""
    # Validate inputs
    if not choices:
        return {"error": "choices list cannot be empty"}
    
    if weights:
        # Validate weights
        if len(weights) != len(choices):
            return {
                "error": f"weights length ({len(weights)}) must match choices length ({len(choices)})",
                "choices": choices,
                "weights": weights
            }
        
        # Check for negative weights
        if any(w < 0 for w in weights):
            return {
                "error": "weights cannot be negative",
                "choices": choices,
                "weights": weights
            }
        
        # Validate sum (allow small floating point error)
        weight_sum = sum(weights)
        if abs(weight_sum - 1.0) > 0.001:
            return {
                "error": f"weights must sum to 1.0, got {weight_sum}",
                "choices": choices,
                "weights": weights
            }
        
        result = random.choices(choices, weights=weights, k=1)[0]
    else:
        result = random.choice(choices)
    
    return {"result": result, "choices": choices, "weights": weights}

# ============================================================
# PASTE EXPORTED CONVERSATION HERE (or use sample below)
# ============================================================
conversation = [
    {
        "role": "system",
        "content": """You are a game master for an interactive story. Use the provided tools for:
- calculator_pemdas: All math calculations (damage, resources, etc.)
- random_number_given_min_max: Dice rolls, random events
- random_choice_with_weights: Outcome determination with probabilities

Always show your calculations explicitly and consider counterfactuals before determining outcomes."""
    },
    {
        "role": "user",
        "content": """I'm exploring a dungeon. I have 50 gold and 20 health.

I encounter a goblin. I attack with my sword (base damage 10-15). 

What happens?"""
    }
]

# ============================================================
# SET MODEL HERE
# ============================================================
model = "anthropic/claude-sonnet-4.5"  
# Options:
#   "anthropic/claude-sonnet-4.5" (pro tier storytelling, best reasoning)
#   "anthropic/claude-haiku-4.5" (budget tier storytelling, fast)
#   "google/gemini-2.5-pro" (pro tier management)
#   "google/gemini-2.5-flash" (budget tier management)

# ============================================================
# RUN TOOL CALLING LOOP
# ============================================================
print(f"🚀 Testing {model} with tool calling...\n")
print("="*80)

messages = conversation.copy()
max_iterations = 10  # Prevent infinite loops

for iteration in range(max_iterations):
    # Send request to OpenRouter
    payload = {
        "model": model,
        "messages": messages,
        "tools": tools,
        "reasoning": {"effort": "high"},
        "temperature": 0,
        "top_p": 0.8
        "max_tokens": 4000
    }
    
    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        },
        json=payload
    )
    
    result = response.json()
    
    if 'error' in result:
        print(f"❌ Error: {result['error']}")
        break
    
    assistant_message = result['choices'][0]['message']
    choice_data = result['choices'][0]
    
    # Extract thinking/reasoning content from reasoning_details
    # Check both assistant_message and choice_data
    reasoning_text = ""
    
    # First check assistant_message for reasoning_details
    if 'reasoning_details' in assistant_message:
        reasoning_details = assistant_message.get('reasoning_details', [])
        for detail in reasoning_details:
            if detail.get('type') == 'reasoning.text' and 'text' in detail:
                reasoning_text = detail.get('text', '')
                break
    
    # Also check choice_data level
    if not reasoning_text and 'reasoning_details' in choice_data:
        reasoning_details = choice_data.get('reasoning_details', [])
        for detail in reasoning_details:
            if detail.get('type') == 'reasoning.text' and 'text' in detail:
                reasoning_text = detail.get('text', '')
                break
    
    # Fallback to direct reasoning field
    if not reasoning_text and 'reasoning' in assistant_message:
        reasoning_text = assistant_message.get('reasoning', '')
    
    # Print thinking if found
    if reasoning_text:
        print(f"\n🧠 THINKING (Iteration {iteration + 1}):")
        print(reasoning_text)
        print()
    
    messages.append(assistant_message)
    
    # Check if there are tool calls
    if assistant_message.get('tool_calls'):
        print(f"\n🔧 Tool Calls (Iteration {iteration + 1}):")
        
        for tool_call in assistant_message['tool_calls']:
            tool_name = tool_call['function']['name']
            tool_args = json.loads(tool_call['function']['arguments'])
            tool_id = tool_call['id']
            
            print(f"  - {tool_name}({json.dumps(tool_args, indent=2)})")
            
            # Execute the tool
            if tool_name == "calculator_pemdas":
                tool_result = execute_calculator_pemdas(tool_args['expression'])
            elif tool_name == "random_number_given_min_max":
                tool_result = execute_random_number_given_min_max(tool_args['min'], tool_args['max'])
            elif tool_name == "random_choice_with_weights":
                tool_result = execute_random_choice_with_weights(
                    tool_args['choices'],
                    tool_args.get('weights')
                )
            else:
                tool_result = {"error": f"Unknown tool: {tool_name}"}
            
            print(f"    Result: {json.dumps(tool_result, indent=2)}")
            
            # Add tool result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_id,
                "content": json.dumps(tool_result)
            })
        
        # Continue loop to get final response
        continue
    
    # No more tool calls, show final response
    print(f"\n📝 FINAL RESPONSE:\n")
    print(assistant_message.get('content', ''))
    break

print("\n" + "="*80)

# Show token usage
if 'usage' in result:
    usage = result['usage']
    total = usage.get('total_tokens', 0)
    prompt = usage.get('prompt_tokens', 0)
    completion = usage.get('completion_tokens', 0)
    reasoning_tokens = usage.get('reasoning_tokens', 0)
    token_str = f"\n📊 Tokens: {total} total ({prompt} prompt + {completion} completion"
    if reasoning_tokens > 0:
        token_str += f" + {reasoning_tokens} reasoning"
    token_str += ")"
    print(token_str)

# Show raw response structure for debugging (includes reasoning_details if present)
print("\n" + "="*80)
print("\n🔍 RAW RESPONSE STRUCTURE (last iteration):\n")
print(json.dumps(result, indent=2))

# Show full conversation
print("\n" + "="*80)
print("\n💬 FULL CONVERSATION:\n")
for i, msg in enumerate(messages, 1):
    role = msg['role'].upper()
    if msg['role'] == 'tool':
        print(f"{i}. TOOL RESULT:")
        try:
            tool_content = json.loads(msg.get('content', '{}'))
            print(json.dumps(tool_content, indent=2))
        except:
            print(msg.get('content', ''))
    elif msg.get('tool_calls'):
        print(f"{i}. {role}:")
        print("  Tool Calls:")
        for tool_call in msg['tool_calls']:
            tool_name = tool_call['function']['name']
            tool_args = json.loads(tool_call['function']['arguments'])
            print(f"    - {tool_name}({json.dumps(tool_args, indent=4)})")
        
        # Check for reasoning_details first
        reasoning_text = ""
        if 'reasoning_details' in msg:
            reasoning_details = msg.get('reasoning_details', [])
            for detail in reasoning_details:
                if detail.get('type') == 'reasoning.text' and 'text' in detail:
                    reasoning_text = detail.get('text', '')
                    break
        
        # Fallback to direct reasoning field
        if not reasoning_text and 'reasoning' in msg:
            reasoning_text = msg.get('reasoning', '')
        
        if reasoning_text:
            print(f"  Reasoning: {reasoning_text}")
        
        if msg.get('content'):
            print(f"  Content: {msg.get('content')}")
    else:
        content = msg.get('content', '')
        print(f"{i}. {role}:")
        if content:
            print(content)
        
        # Check for reasoning_details first
        reasoning_text = ""
        if 'reasoning_details' in msg:
            reasoning_details = msg.get('reasoning_details', [])
            for detail in reasoning_details:
                if detail.get('type') == 'reasoning.text' and 'text' in detail:
                    reasoning_text = detail.get('text', '')
                    break
        
        # Fallback to direct reasoning field
        if not reasoning_text and 'reasoning' in msg:
            reasoning_text = msg.get('reasoning', '')
        
        if reasoning_text:
            print(f"\n  Reasoning: {reasoning_text}")
    print()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (586601715.py, line 249)

In [9]:
messages

[{'role': 'system',
  'content': 'You are a game master for an interactive story. Use the provided tools for:\n- calculator_pemdas: All math calculations (damage, resources, etc.)\n- random_number_given_min_max: Dice rolls, random events\n- random_choice_with_weights: Outcome determination with probabilities\n\nAlways show your calculations explicitly and consider counterfactuals before determining outcomes.'},
 {'role': 'user',
  'content': "I'm exploring a dungeon. I have 50 gold and 20 health.\n\nI encounter a goblin. I attack with my sword (base damage 10-15). \n\nWhat happens?"},
 {'role': 'assistant',
  'content': 'Exciting! Let me resolve your attack on the goblin.\n\n**Rolling your sword damage:**',
  'refusal': None,
  'reasoning': "The user is playing a dungeon exploration game. They want to attack a goblin with their sword. Let me break this down:\n\n1. First, I need to roll for the sword damage (10-15)\n2. Then I need to determine what happens - does the goblin die? Does it